In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import pickle
import patsy
import numpy as np
import pickle
from xopen import xopen

In [12]:
data = pd.read_csv("../../data/PYD_simulation_results.csv.gz", compression='gzip', sep="\t")

In [13]:
data.columns

Index(['genome', 'readlength', 'damage', 'simuCov', 'simuContigLength',
       'reference', 'contiglength', 'GCcontent', 'medianRL', 'null_model_p0',
       'null_model_p0_stdev', 'damage_model_p', 'damage_model_p_stdev',
       'damage_model_pmin', 'damage_model_pmin_stdev', 'damage_model_pmax',
       'damage_model_pmax_stdev', 'pvalue', 'qvalue', 'RMSE',
       'nb_reads_aligned', 'coverage', 'actualCov', 'CtoT-0', 'CtoT-1',
       'CtoT-2', 'CtoT-3', 'CtoT-4', 'GtoA-0', 'GtoA-1', 'GtoA-2', 'GtoA-3',
       'GtoA-4'],
      dtype='object')

In [14]:
data.dtypes

genome                      object
readlength                  object
damage                     float64
simuCov                     object
simuContigLength            object
reference                   object
contiglength                 int64
GCcontent                  float64
medianRL                     int64
null_model_p0              float64
null_model_p0_stdev        float64
damage_model_p             float64
damage_model_p_stdev       float64
damage_model_pmin          float64
damage_model_pmin_stdev    float64
damage_model_pmax          float64
damage_model_pmax_stdev    float64
pvalue                     float64
qvalue                     float64
RMSE                       float64
nb_reads_aligned             int64
coverage                   float64
actualCov                  float64
CtoT-0                     float64
CtoT-1                     float64
CtoT-2                     float64
CtoT-3                     float64
CtoT-4                     float64
GtoA-0              

In [15]:
print(data.columns)

Index(['genome', 'readlength', 'damage', 'simuCov', 'simuContigLength',
       'reference', 'contiglength', 'GCcontent', 'medianRL', 'null_model_p0',
       'null_model_p0_stdev', 'damage_model_p', 'damage_model_p_stdev',
       'damage_model_pmin', 'damage_model_pmin_stdev', 'damage_model_pmax',
       'damage_model_pmax_stdev', 'pvalue', 'qvalue', 'RMSE',
       'nb_reads_aligned', 'coverage', 'actualCov', 'CtoT-0', 'CtoT-1',
       'CtoT-2', 'CtoT-3', 'CtoT-4', 'GtoA-0', 'GtoA-1', 'GtoA-2', 'GtoA-3',
       'GtoA-4'],
      dtype='object')


In [16]:
data = data[['qvalue','damage_model_pmax','coverage','contiglength']]

In [17]:
data = data.loc[data['qvalue'].notna(),:]

## Defining `sig` categorial variable
`True` if `qvalue`<0.05  
`False` if `qvalue`>= 0.05

In [18]:
data['sig'] = pd.cut(data['qvalue'],[0, 0.05, 1], labels=[True,False], include_lowest=True)

In [19]:
data

,qvalue,damage_model_pmax,coverage,contiglength,sig
1896,0.46821,0.17678,3.23761,3068,False
2186,1.00000,0.00007,4.04900,22469,False
2671,1.00000,0.17678,7.22360,3068,False
2985,1.00000,0.01588,7.58392,22469,False
3483,1.00000,0.17678,13.59224,3068,False
...,...,...,...,...,...
701995,0.00000,0.30366,201.63903,346944,True
701996,0.00000,0.30444,260.31416,273518,True
701997,0.00000,0.30184,310.00277,232161,True
701998,0.00000,0.30351,209.11812,287978,True


In [20]:
data.dtypes

qvalue                float64
damage_model_pmax     float64
coverage              float64
contiglength            int64
sig                  category
dtype: object

In [21]:
data = data.drop('qvalue', axis=1)

In [22]:
data.rename(columns={'damage_model_pmax':'damage'}, inplace=True)

### Reordering the categories

## Model formula

In [23]:
formula = 'sig ~ coverage + damage + contiglength'

## Creating the GLM logistic model

In [24]:
model_call = smf.glm(formula= formula, data=data, family=sm.families.Binomial())

In [25]:
model = model_call.fit()

In [26]:
print(model.summary())

                      Generalized Linear Model Regression Results                      
Dep. Variable:     ['sig[True]', 'sig[False]']   No. Observations:               610730
Model:                                     GLM   Df Residuals:                   610726
Model Family:                         Binomial   Df Model:                            3
Link Function:                           logit   Scale:                          1.0000
Method:                                   IRLS   Log-Likelihood:            -2.0981e+05
Date:                         Wed, 17 Mar 2021   Deviance:                   4.1963e+05
Time:                                 20:40:24   Pearson chi2:                 8.62e+11
No. Iterations:                              9                                         
Covariance Type:                     nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------

### Saving the model

In [27]:
model.save("../../models/accuracy_model_v2_python.pickle", remove_data=True)

/projects1/users/borry/15_miniconda3/envs/pydamage/lib/python3.7/site-packages/statsmodels/genmod/generalized_linear_model.py:1561: FutureWarning: Anscombe residuals currently unscaled. After the 0.12 release, they will be scaled.
  'release, they will be scaled.', category=FutureWarning)


In [28]:
! gzip ../../models/accuracy_model_v2_python.pickle

### Test data

In [67]:
d = pd.Series([10,10000,0.05]).to_frame(name='NZ_JHCB02000014.1').transpose()
d = pd.DataFrame([range(1,41),[10000]*40,[0.2]*40]).transpose()

In [68]:
d.columns = ['coverage','contiglength','damage']

In [69]:
d

,coverage,contiglength,damage
0,1.0,10000.0,0.2
1,2.0,10000.0,0.2
2,3.0,10000.0,0.2
3,4.0,10000.0,0.2
4,5.0,10000.0,0.2
5,6.0,10000.0,0.2
6,7.0,10000.0,0.2
7,8.0,10000.0,0.2
8,9.0,10000.0,0.2
9,10.0,10000.0,0.2


### Making inference on test data

In [70]:
model.predict(d)

0     0.582730
1     0.588989
2     0.595220
3     0.601420
4     0.607588
5     0.613722
6     0.619820
7     0.625880
8     0.631901
9     0.637881
10    0.643819
11    0.649713
12    0.655561
13    0.661363
14    0.667117
15    0.672821
16    0.678475
17    0.684076
18    0.689625
19    0.695120
20    0.700560
21    0.705944
22    0.711271
23    0.716540
24    0.721751
25    0.726902
26    0.731993
27    0.737024
28    0.741994
29    0.746902
30    0.751748
31    0.756531
32    0.761251
33    0.765909
34    0.770502
35    0.775032
36    0.779499
37    0.783901
38    0.788239
39    0.792514
dtype: float64

Opening model from file

In [33]:
with xopen("../../models/accuracy_model_v2_python.pickle.gz", 'rb') as mod_p:
    model2 = pickle.load(mod_p)

In [34]:
model2.predict(d)

NZ_JHCB02000014.1    0.949145
dtype: float64